In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [3]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
inspector = inspect(engine)

In [4]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [6]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [7]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [8]:
# Get column names and types for reference
inspector.get_columns("measurement")
columns = inspector.get_columns("measurement")
for column in columns:  
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [9]:
# Preview table
meas_preview = engine.execute('SELECT * FROM measurement LIMIT 1')
for row in meas_preview:
    print(row)

(1, 'USC00519397', '2010-01-01', 0.08, 65.0)


In [10]:
# Design a query to retrieve the last 12 months of precipitation data.
# Retrieve the most recent date in the dataset to calculate the last 12 months
last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
# Retrieve the last 12 months of precipitation data
last_12_mo_data = session.query(Measurement.id, Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs).filter(Measurement.date <= '2017-08-23').filter(Measurement.date >= '2016-08-23').all()
prcp_and_date_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date <= '2017-08-23').filter(Measurement.date >= '2016-08-23').all()

In [11]:
#Load the query results into a Pandas DataFrame and set the index to the date column
prcp_df = pd.DataFrame(prcp_and_date_data, columns=['date', 'precipitation'])
prcp_df.set_index('date', inplace=True)
prcp_df.head()

,precipitation
date,
2016-08-23,0.00
2016-08-24,0.08
2016-08-25,0.08
2016-08-26,0.00
2016-08-27,0.00


In [12]:
# Sort the DataFrame Values by date
prcp_df.sort_values(by=['date'])
prcp_df.head()

,precipitation
date,
2016-08-23,0.00
2016-08-24,0.08
2016-08-25,0.08
2016-08-26,0.00
2016-08-27,0.00


In [ ]:
# Plot the results using the DataFrame plot method
prcp_df.plot.bar()
plt.tight_layout()
plt.show()

In [ ]:
prcp_df.describe

In [ ]:
# Get column names and types for reference
inspector.get_columns("Station")
columns = inspector.get_columns("Station")
for column in columns:  
    print(column["name"], column["type"])

In [ ]:
# Design a query to calculate the total number of stations
total_stations = session.query(Station.station).count()
print(total_stations)

In [ ]:
# Design a query to find the most active stations
most_active_stations = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc())
most_active_stations.all()

# Which station has the highest number of observations?
# Answer: USC00519281

In [ ]:
# Design a query to retrieve the last 12 months of temperature observation data (tobs)
tobs_last_12_mo = session.query(Measurement.date, Measurement.tobs).group_by(Measurement.date).filter(Measurement.date <= '2017-08-23').filter(Measurement.date >= '2016-08-23').all()
# Filter by the highest number of observations
tobs_last_12_mo = session.query(Measurement.date, Measurement.tobs).group_by(Measurement.date).filter(Measurement.date <= '2017-08-23').filter(Measurement.date >= '2016-08-23').filter(Measurement.station=='USC00519281').all()

In [ ]:
# Plot the results as a histogram with bins=12
stations_hist = pd.DataFrame(data=tobs_last_12_mo, columns=["station", "date", "tobs"])
stations_hist = stations_hist.set_index('date', drop=True)
stations_hist = stations_hist.drop(columns="station", axis=1)

x = stations_hist['tobs']
num_bins = 12
plt.figure(figsize=[10, 7])
temp_plot = plt.hist(x, num_bins, color='lightskyblue', label='tobs', alpha=.8)


plt.ylabel('Frequency', fontsize=10)
plt.tick_params(axis='y', labelsize=10)
plt.tick_params(axis='x', labelsize=10)
legend = plt.legend(frameon=True, edgecolor='black', fontsize='large')

In [ ]:
# STEP 2

In [ ]:
from flask import Flask

In [ ]:
# Routes
# List all routes available
@app.route('/')
def homepage():
    return(
        f'Available Routes:'
        f'Date range is from 2010-01-01 through 2017-08-23.'

        f'/api/v1.0/precipitation'
        f'Query dates and temperature from the last year.''

        f'/api/v1.0/stations'
        f'Returns a JSON list of stations from the dataset.'

        f'/api/v1.0/tobs'
        f'Return a JSON list of Temperature Observations (tobs) for the previous year.'

        f'/api/v1.0/<start> and /api/v1.0/<start>/<end>'
        f'Returns a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.'
        )

In [ ]:
# Return the JSON representation of your dictionary.
@app.route('/api/v1.0/precipitation')
def precipitation():
    prcp_results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date <= '2017-08-23').filter(Measurement.date >= '2016-08-23').all()
    prcp_results_list = []
    for prcp in prc_results
        prcp_dict = {}
        prcp_dict["date"] = result[0]
        prcp_dict["prcp"] = float(result[1])
        prcp_results_list.append(prcp_dict)

    return jsonify(prcp_results_list)

In [ ]:
# Return a JSON list of stations from the dataset.
@app.route('/api/v1.0/stations')
def stations():
    all_stations = session.query(Station.station).all()
    all_stations_list = list(np.ravel(all_stations))
    
    return jsonify(all_stations_list)

In [ ]:
# Return a JSON list of Temperature Observations (tobs) for the previous year.
@app.route('/api/v1.0/tobs')
def tobs():
    all_tobs = session.query(Measurement.date, Measurement.tobs).group_by(Measurement.date).filter(Measurement.date <= '2017-08-23').filter(Measurement.date >= '2016-08-23').all()
    all_tobs_list = list(np.ravel(all_tobs))
    return jsonify(all_tobs_list)

In [ ]:
#Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.
def start_end(start=None, end=None):
    # When given the start only, calculate TMIN, TAVG, and TMAX for all dates greater than and equal to the start date.
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    if not end:
        temps1 = session.query(*sel).filter(Measurement.date >= start).all()
        temps1_list = list(np.ravel(temps1))
        return jsonify(temps1_list)
    
    # When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start and end date inclusive.
    temps2 = session.query(*sel).filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    temps2_list = list(np.ravel(temps2))
    return jsonify(temps2_list)

if __name__ == '__main__':
    app.run()